#### INSTALL SPARK

In [1]:
# install conda,spark,java,winutils and set correct environment varaibles - https://www.youtube.com/watch?v=DznteGdeJoA
import findspark 
findspark.init()
findspark.find()

'C:\\spark-3.2.0-bin-hadoop3.2'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, lit, trim, ltrim, rtrim, split, regexp_extract, expr, coalesce

# entry point for spark's dataframes
spark = SparkSession.builder \
    .master("local") \
    .appName("pharmacy scraper") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:3.2.0_0.16.0") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

In [3]:
spark

### 1. DATA CLEANSING - transform data in such a way that each branch represents a single row

#### EXTRACT

In [4]:
# convert excel file into spark's dataframe 
df_generika = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/generika.xlsx")
df_tgp = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/tgp.xlsx")
df_watsons = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/watsons.xlsx")

df_rose_luzon = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/rosepharmacy - luzon.xlsx")
df_rose_visayas = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/rosepharmacy - visayas.xlsx")
df_rose_mindanao = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/rosepharmacy - mindanao.xlsx")

df_mercury_luzon = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/mercury - luzon.xlsx")
df_mercury_metro_manila = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/mercury - metro manila.xlsx")
df_mercury_visayas = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/mercury - visayas.xlsx")
df_mercury_mindanao = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/mercury - mindanao.xlsx")

df_ssd_name = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/ssd-name.xlsx")
df_ssd_others = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "Sheet1").load("./../data/raw-data/ssd-address_storeHours_phone_removeHrTag.xlsx")

#### TRANSFORM generika

In [5]:
# quick inspection of generika's dataframe
df_generika.createOrReplaceTempView("generika")
initial_inspect = spark.sql("SELECT * FROM generika")

initial_inspect.show()

+--------------------+-------------------+--------------------+--------------------+
|       store-address| store-address href|     store-address 2|        store-holder|
+--------------------+-------------------+--------------------+--------------------+
|                null|               null|                null|                null|
|CVT BACOOR E. ST....|javascript:void(0);|Evangelista Stree...|0917-8624935 / 43...|
|LGN STAROSA BALIBAGO|javascript:void(0);|1360 Old National...|        0917-8808883|
|           NAGA CITY|javascript:void(0);|622 Penafrancia A...|0917-8718924 / 05...|
|   BATS RIZAL AVENUE|javascript:void(0);|9 Rizal Avenue, B...|0917-8415926 / 04...|
|                    |                   |                    |                    |
|   QUEZON, CATANAUAN|javascript:void(0);|Rayo de Sol Bldg....|        0917-8600313|
|  CVT DASMA DISTRICT|javascript:void(0);|Space No. LG-20, ...|0917-8632302 / 04...|
| BATS ROSARIO QUILIB|javascript:void(0);|Medical Expert Bl...|09

In [6]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_generika.select([count(when(col(c).isNull(), c)).alias(c) for c in df_generika.columns])
nan_values = df_generika.select([count(when(isnan(c), c)).alias(c) for c in df_generika.columns])
empty_values = df_generika.select([count(when(col(c) == "", c)).alias(c) for c in df_generika.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+-------------+------------------+---------------+------------+
|store-address|store-address href|store-address 2|store-holder|
+-------------+------------------+---------------+------------+
|            4|                 4|              4|           4|
|            0|                 0|              0|           0|
|          200|               200|            200|         312|
+-------------+------------------+---------------+------------+



In [7]:
# remove - useless column from the selection, null rows, empty row | note: update and delete isn't supported in spark
remove_cleaning_generika = spark.sql("""
    SELECT ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS row,`store-address 2`, `store-holder` 
    FROM generika 
    WHERE `store-address` IS NOT NULL AND NOT `store-address` = ""
""")

# add new column - store name 
add_name_cleaning_generika = remove_cleaning_generika.withColumn("Name", lit("Generika"))

# fill na values for phone with 0 | note: update and delete isn't supported in spark - so we have to do them with spark's rdd
fill_null_cleaning_generika = add_name_cleaning_generika.rdd.map(lambda row: (row[0], row[1], 0 if row[2] == "" else row[2], row[3]))
fill_null_cleaning_generika.collect()

# create a new tempview from the cleaned dataframe 
cleaned_generika = fill_null_cleaning_generika.toDF(["Row","Address","Contact", "Name"])
cleaned_generika.select("Row","Address","Contact","Name").show()
cleaned_generika.createOrReplaceTempView("cleaned_generika")

+---+--------------------+--------------------+--------+
|Row|             Address|             Contact|    Name|
+---+--------------------+--------------------+--------+
|  1|Evangelista Stree...|0917-8624935 / 43...|Generika|
|  2|1360 Old National...|        0917-8808883|Generika|
|  3|622 Penafrancia A...|0917-8718924 / 05...|Generika|
|  4|9 Rizal Avenue, B...|0917-8415926 / 04...|Generika|
|  5|Rayo de Sol Bldg....|        0917-8600313|Generika|
|  6|Space No. LG-20, ...|0917-8632302 / 04...|Generika|
|  7|Medical Expert Bl...|0917-6537444 / 04...|Generika|
|  8|Market Site, Brgy...|        0917-8940065|Generika|
|  9|TGB Bldg., Iguald...|0917-8444280 / 05...|Generika|
| 10|Molino 2, Bacoor ...|        0917-8365290|Generika|
| 11|Brgy. Sta. Maria,...|        0917-8376353|Generika|
| 12|Phase 3 Block 112...|0917-5043871 / 09...|Generika|
| 13|Casa De Ida Bldg....|        0917-8392092|Generika|
| 14|No. 0489, Boot, T...|        0917-8089986|Generika|
| 15|Villa Grande Home...|     

In [8]:
# final inspection for generika 
null_values = cleaned_generika.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_generika.columns])
nan_values = cleaned_generika.select([count(when(isnan(c), c)).alias(c) for c in cleaned_generika.columns])
empty_values = cleaned_generika.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_generika.columns])

final_inspect = null_values.union(nan_values).union(empty_values)
final_inspect.show()

+---+-------+-------+----+
|Row|Address|Contact|Name|
+---+-------+-------+----+
|  0|      0|      0|   0|
|  0|      0|      0|   0|
|  0|      0|      0|   0|
+---+-------+-------+----+



#### TRANSFORM tgp

In [9]:
# quick inspection of tgp's dataframe
df_tgp.createOrReplaceTempView("tgp")
initial_inspect = spark.sql("SELECT * FROM tgp")

initial_inspect.show()

+--------------------+--------------------+--------------------+------------+
|                Name|             Address|                City|       Phone|
+--------------------+--------------------+--------------------+------------+
|TGP The Generics ...|Osmena St. Poblac...|   North Cotabato,  |0917-7040504|
|TGP The Generics ...|Poblacion Carmen,...|   North Cotabato,  |           0|
|TGP The Generics ...|Poblacion Datu Od...|      Maguindanao,  |  9069743889|
|TGP The Generics ...|Mabini Corner E. ...|   South Cotabato,  |  9437025959|
|TGP The Generics ...|Orocio Bldg., Alu...|   South Cotabato,  |  9228302752|
|TGP The Generics ...|Poblacion 8 Midsa...|   North Cotabato,  |     5230357|
|TGP The Generics ...|Rizal Avenue, Pob...|   North Cotabato,  |   645726748|
|TGP The Generics ...|Poblacion 8 Midsa...|   North Cotabato,  |     5230357|
|TGP The Generics ...|Prk. 5 Poblacion ...|Compostela Valley,  |  9357984426|
|TGP The Generics ...|Cor. Rizal & Arel...|  Davao Del Norte,  |

In [10]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_tgp.select([count(when(col(c).isNull(), c)).alias(c) for c in df_tgp.columns])
nan_values = df_tgp.select([count(when(isnan(c), c)).alias(c) for c in df_tgp.columns])
empty_values = df_tgp.select([count(when(col(c) == "", c)).alias(c) for c in df_tgp.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+----+-------+----+-----+
|Name|Address|City|Phone|
+----+-------+----+-----+
|   0|      0|   0|    0|
|   0|      0|   0|    0|
|   0|      0|   0|    0|
+----+-------+----+-----+



In [11]:
# remove city 
remove_cleaning_tgp = spark.sql("""
    SELECT ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row, Address, Phone as Contact
    FROM tgp
""")

cleaned_tgp = remove_cleaning_tgp
cleaned_tgp = cleaned_tgp.withColumn("Name", lit("The Generics Pharmacy"))
cleaned_tgp.show()

+---+--------------------+------------+--------------------+
|Row|             Address|     Contact|                Name|
+---+--------------------+------------+--------------------+
|  1|Osmena St. Poblac...|0917-7040504|The Generics Phar...|
|  2|Poblacion Carmen,...|           0|The Generics Phar...|
|  3|Poblacion Datu Od...|  9069743889|The Generics Phar...|
|  4|Mabini Corner E. ...|  9437025959|The Generics Phar...|
|  5|Orocio Bldg., Alu...|  9228302752|The Generics Phar...|
|  6|Poblacion 8 Midsa...|     5230357|The Generics Phar...|
|  7|Rizal Avenue, Pob...|   645726748|The Generics Phar...|
|  8|Poblacion 8 Midsa...|     5230357|The Generics Phar...|
|  9|Prk. 5 Poblacion ...|  9357984426|The Generics Phar...|
| 10|Cor. Rizal & Arel...|  9172051698|The Generics Phar...|
| 11|Stall 112 2 Store...|  9273926022|The Generics Phar...|
| 12|Blk. 6 Stall B Pu...|  9055883230|The Generics Phar...|
| 13|Dr. 3 Laureano Pe...| 91553633275|The Generics Phar...|
| 14|P-13 Poblacion Na..

In [12]:
# final inspection for tgp
null_values = cleaned_tgp.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_tgp.columns])
nan_values = cleaned_tgp.select([count(when(isnan(c), c)).alias(c) for c in cleaned_tgp.columns])
empty_values = cleaned_tgp.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_tgp.columns])

final_inspect = null_values.union(nan_values).union(empty_values)
final_inspect.show()

+---+-------+-------+----+
|Row|Address|Contact|Name|
+---+-------+-------+----+
|  0|      0|      0|   0|
|  0|      0|      0|   0|
|  0|      0|      0|   0|
+---+-------+-------+----+



#### TRANSFORM watsons

In [13]:
# quick inspection of watsons' dataframe
df_watsons.createOrReplaceTempView("watsons")
initial_inspect = spark.sql("SELECT * FROM watsons")

initial_inspect.show()

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|store-order|          store-name|             address|                time|                 tel|            tel href|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          1|WATSONS SM CUBAO ...|Quezon City, UGF ...|FRI-SAT  10:00 AM...|(02) 8-911-3661 /...|tel:(02) 8-911-36...|
|          2|WATSONS SM QUIAPO...|Manila City, UGF ...|MON-SUN  10:00 AM...|(02) 8-733-8416 t...|tel:(02) 8-733-84...|
|          3|WATSONS SM MAKATI...|Makati City, UGF ...|FRI-SAT  10:00 AM...|(02) 8-816-6421 /...|tel:(02) 8-816-64...|
|          4|WATSONS SM CITY C...|Quezon City, UGF ...|MON-SUN  10:00 AM...|     (02) 8-716-3718| tel:(02) 8-716-3718|
|          5|WATSONS SM CITY N...|Quezon City, UGF ...|MON-SUN  10:00 AM...|(02) 3-453-3154 /...|tel:(02) 3-453-31...|
|          6|WATSONS SM MEGAMA...|Mandaluyong Ci

In [14]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_watsons.select([count(when(col(c).isNull(), c)).alias(c) for c in df_watsons.columns])
nan_values = df_watsons.select([count(when(isnan(c), c)).alias(c) for c in df_watsons.columns])
empty_values = df_watsons.select([count(when(col(c) == "", c)).alias(c) for c in df_watsons.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+-----------+----------+-------+----+---+--------+
|store-order|store-name|address|time|tel|tel href|
+-----------+----------+-------+----+---+--------+
|          0|         0|      0|   0|  0|       0|
|          0|         0|      0|   0|  0|       0|
|          0|         0|      0|   0|  0|       0|
+-----------+----------+-------+----+---+--------+



In [15]:
# rename columns, remove tel href (duplicate)
remove_cleaning_watsons = spark.sql("""
    SELECT `store-order` AS Row, `address` AS Address, TRIM(tel) AS Contact
    FROM watsons
""")

cleaned_watsons = remove_cleaning_watsons
cleaned_watsons = cleaned_watsons.withColumn("Name", lit("Watsons Pharmacy"))
cleaned_watsons.show()

+---+--------------------+--------------------+----------------+
|Row|             Address|             Contact|            Name|
+---+--------------------+--------------------+----------------+
|  1|Quezon City, UGF ...|(02) 8-911-3661 /...|Watsons Pharmacy|
|  2|Manila City, UGF ...|(02) 8-733-8416 t...|Watsons Pharmacy|
|  3|Makati City, UGF ...|(02) 8-816-6421 /...|Watsons Pharmacy|
|  4|Quezon City, UGF ...|     (02) 8-716-3718|Watsons Pharmacy|
|  5|Quezon City, UGF ...|(02) 3-453-3154 /...|Watsons Pharmacy|
|  6|Mandaluyong City,...|     (02) 8-687-4296|Watsons Pharmacy|
|  7|Cavite, UGF SM Ci...|(046) 417-0869 / ...|Watsons Pharmacy|
|  8|Quezon City, UGF ...|(02) 3-417-6714 /...|Watsons Pharmacy|
|  9|Outlet 1 SM City ...|    (046) 3-417-5334|Watsons Pharmacy|
| 10|Cavite, Outlet 2 ...|    (046) 3-417-2400|Watsons Pharmacy|
| 11|Quezon City, Outl...|     (02) 8-936-8415|Watsons Pharmacy|
| 12|Cavite, Supermark...|    (046) 3-417-3695|Watsons Pharmacy|
| 13|Muntinlupa City, ...

In [16]:
# final inspection for watsons
cleaned_watsons.show(5)

+---+--------------------+--------------------+----------------+
|Row|             Address|             Contact|            Name|
+---+--------------------+--------------------+----------------+
|  1|Quezon City, UGF ...|(02) 8-911-3661 /...|Watsons Pharmacy|
|  2|Manila City, UGF ...|(02) 8-733-8416 t...|Watsons Pharmacy|
|  3|Makati City, UGF ...|(02) 8-816-6421 /...|Watsons Pharmacy|
|  4|Quezon City, UGF ...|     (02) 8-716-3718|Watsons Pharmacy|
|  5|Quezon City, UGF ...|(02) 3-453-3154 /...|Watsons Pharmacy|
+---+--------------------+--------------------+----------------+
only showing top 5 rows



#### TRANSFORM rosepharmacy - luzon

In [17]:
# quick inspection of rosepharmacy - luzon's dataframe
df_rose_luzon.createOrReplaceTempView("rose_luzon")
initial_inspect = spark.sql("""
    SELECT 
        *, 
        ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row 
    FROM rose_luzon 
""")

initial_inspect.show(20)

+-------------------------+---------------------------+---+
|tablescraper-selected-row|tablescraper-selected-row 2|Row|
+-------------------------+---------------------------+---+
|     1st Flr, Festival...|       Level 1 Festival ...|  1|
|     Monday – Sunday 9...|              Store Hours :|  2|
|     Zapote Road, Fili...|       South Station Ala...|  3|
|     Monday – Sunday 6...|              Store Hours :|  4|
|     G/F Asian Hospita...|       Asian Hospital Al...|  5|
|     Monday – Sunday 7...|              Store Hours :|  6|
|     space 2102A & 210...|       Level 2 Festival ...|  7|
|     Monday – Sunday 9...|              Store Hours :|  8|
|     Sucat, Muntinlupa...|       Lakefront Broadwa...|  9|
|     Monday – Sunday 9...|              Store Hours :| 10|
|     Calamba Arcade, N...|             Calamba Laguna| 11|
|     Monday – Sunday 6...|              Store Hours :| 12|
|     P. Torres St., Li...|       Torres Lipa City ...| 13|
|     Monday – Sunday 6...|             

In [18]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_rose_luzon.select([count(when(col(c).isNull(), c)).alias(c) for c in df_rose_luzon.columns])
nan_values = df_rose_luzon.select([count(when(isnan(c), c)).alias(c) for c in df_rose_luzon.columns])
empty_values = df_rose_luzon.select([count(when(col(c) == "", c)).alias(c) for c in df_rose_luzon.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+-------------------------+---------------------------+
|tablescraper-selected-row|tablescraper-selected-row 2|
+-------------------------+---------------------------+
|                        0|                          0|
|                        0|                          0|
|                        1|                          1|
+-------------------------+---------------------------+



In [19]:
# inspect further the empty row 
# 1. find the row number for the empty row 
initial_inspect.createOrReplaceTempView("initial_inspect_rose_luzon")
further_inspect = spark.sql("""
    SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM initial_inspect_rose_luzon
    WHERE `tablescraper-selected-row` = "" OR `tablescraper-selected-row 2` = ""
""")

# 2. show rows before and after empty rows 
more_further_inspect = spark.sql("""
    SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM initial_inspect_rose_luzon
    WHERE `Row` >= 25 AND `Row` <= 31
""").show()

+---+-------------------------+---------------------------+
|Row|tablescraper-selected-row|tablescraper-selected-row 2|
+---+-------------------------+---------------------------+
| 25|     GF Gaisano Capita...|       Calapan Mindoro O...|
| 26|     Monday – Sunday 8...|              Store Hours :|
| 27|     GF-06 Gaisano Cap...|       Gaisano Capital S...|
| 28|                         |       Shopwise Imus Cavite|
| 29|     Shopwise Imus, Em...|                           |
| 30|     Monday – Sunday 9...|              Store Hours :|
| 31|     N. Domingo cor. G...|       San Juan Metro Ma...|
+---+-------------------------+---------------------------+



In [20]:
# add an empty row at row 28 and row 30(previously 29) to balance data distribution
new_row = spark.createDataFrame([("27.5","",""),("28.5","","")], ["Row","tablescraper-selected-row","tablescraper-selected-row 2"])
add_empty_row = spark.sql("SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2` FROM initial_inspect_rose_luzon").unionAll(new_row)
add_empty_row.createOrReplaceTempView("add_empty_row_rose_luzon")
new_added_row = spark.sql("""
    WITH add_empty_row AS (
        SELECT CAST(Row AS INTEGER), `tablescraper-selected-row`, `tablescraper-selected-row 2`
        FROM add_empty_row_rose_luzon
        ORDER BY `Row`
    ), 
    reorder_row AS (
        SELECT *, CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS RowCount
        FROM add_empty_row
    )
    SELECT RowCount as Row, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM reorder_row
""")

# remove the store hours from rows 
new_added_row.createOrReplaceTempView("added_empty_row_rose_luzon")
remove_store_hours_cleaning_rose_luzon = spark.sql("""
    SELECT *
    FROM added_empty_row_rose_luzon
    WHERE `Row` % 2 = 1
""")

# separate contact from address in tablescraper-selected-row column and reorder the row count
remove_store_hours_cleaning_rose_luzon.createOrReplaceTempView("remove_store_hours_rose_luzon")
address_rose_luzon = remove_store_hours_cleaning_rose_luzon.withColumn("Address", split(remove_store_hours_cleaning_rose_luzon['tablescraper-selected-row'], '\n').getItem(0)).createOrReplaceTempView("address_rose_luzon")
contact_rose_luzon = remove_store_hours_cleaning_rose_luzon.withColumn("Contact", split(remove_store_hours_cleaning_rose_luzon['tablescraper-selected-row'], '\n').getItem(1)).createOrReplaceTempView("contact_rose_luzon")
address_contact_cleaning_rose_luzon = spark.sql("""
        SELECT 
            CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, 
            Address, 
            TRIM('Tel. ' FROM Contact) AS Contact
        FROM address_rose_luzon
        JOIN contact_rose_luzon
        ON address_rose_luzon.Row = contact_rose_luzon.Row
        WHERE NOT `Address` = ""
""")

# replace null values with 0
fill_null_cleaning_rose_luzon = address_contact_cleaning_rose_luzon.rdd.map(lambda row: (row[0], row[1], 0 if row[2] == None else row[2]))
fill_null_cleaning_rose_luzon.collect()

cleaned_rose_luzon = fill_null_cleaning_rose_luzon.toDF(["Row","Address","Contact"])
cleaned_rose_luzon.show(20,False)

+---+------------------------------------------------------------------------------------------+--------------+
|Row|Address                                                                                   |Contact       |
+---+------------------------------------------------------------------------------------------+--------------+
|1  |1st Flr, Festival Supermall, Filinvest Corporate Ctiy, Alabang, Muntinlupa City           |(02) 850-2853 |
|2  |Zapote Road, Filinvest Corporate City, Alabang, Muntinlupa City                           |(02) 807-9102 |
|3  |G/F Asian Hospital, Filinvest Corporate City, Alabang, Muntinlupa City                    |(02) 659-3394 |
|4  |space 2102A & 2106, 2nd level Festival Supermall, Filinvest corporate city,Alabang        |(02) 822-9492 |
|5  |Sucat, Muntinlupa                                                                         |(02) 856-0293 |
|6  |Calamba Arcade, National Highway, Crossing, Calamba City Laguna                           |(049) 54

#### TRANSFORM rosepharmacy - visayas

In [21]:
# quick inspection of rosepharmacy - luzon's dataframe
df_rose_visayas.createOrReplaceTempView("rose_visayas")
initial_inspect = spark.sql("""
    SELECT 
        *, 
        ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row 
    FROM rose_visayas
""")

initial_inspect.show(20)

+-------------------------+---------------------------+---+
|tablescraper-selected-row|tablescraper-selected-row 2|Row|
+-------------------------+---------------------------+---+
|     Osmeña Boulevard,...|           Osmeña Boulevard|  1|
|     Monday – Sunday 7...|              Store Hours :|  2|
|     Cinco Centrum, Fu...|              Fuente Osmeña|  3|
|     Monday – Sunday O...|              Store Hours :|  4|
|     Unit 159, Upper G...|               SM City Cebu|  5|
|     Monday – Sunday 1...|              Store Hours :|  6|
|     Stall 277, 2nd Fl...|          Ayala Center Cebu|  7|
|     Monday – Sunday 1...|              Store Hours :|  8|
|     Robinsons Cyberga...|        Robinsons Cybergate|  9|
|     Monday – Sunday 8...|              Store Hours :| 10|
|     F Ramos Street, C...|                  ABC Hotel| 11|
|     Monday – Sunday 7...|              Store Hours :| 12|
|     Diaz Bldg. Cor. A...|                      Avila| 13|
|     Monday – Sunday 7...|             

In [22]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_rose_visayas.select([count(when(col(c).isNull(), c)).alias(c) for c in df_rose_visayas.columns])
nan_values = df_rose_visayas.select([count(when(isnan(c), c)).alias(c) for c in df_rose_visayas.columns])
empty_values = df_rose_visayas.select([count(when(col(c) == "", c)).alias(c) for c in df_rose_visayas.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+-------------------------+---------------------------+
|tablescraper-selected-row|tablescraper-selected-row 2|
+-------------------------+---------------------------+
|                        0|                          0|
|                        0|                          0|
|                        1|                          1|
+-------------------------+---------------------------+



In [23]:
# inspect further the empty row 
# 1. find the row number for the empty row 
initial_inspect.createOrReplaceTempView("initial_inspect_rose_visayas")
further_inspect = spark.sql("""
    SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM initial_inspect_rose_visayas
    WHERE `tablescraper-selected-row` = "" OR `tablescraper-selected-row 2` = ""
""")

# 2. show rows before and after empty rows 
more_further_inspect = spark.sql("""
    SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM initial_inspect_rose_visayas
    WHERE `Row` >= 150 AND `Row` <= 160
""").show()

+---+-------------------------+---------------------------+
|Row|tablescraper-selected-row|tablescraper-selected-row 2|
+---+-------------------------+---------------------------+
|150|     GF M-1 Gaisano Gr...|       Gaisano Grandmall...|
|151|     Monday – Thursday...|              Store Hours :|
|152|     Insular Square,Ta...|       Insular Square Ma...|
|153|     Monday – Thursday...|              Store Hours :|
|154|                         |       Gaisano Saversmar...|
|155|     Gaisano Saversmar...|                           |
|156|     Monday – Thursday...|              Store Hours :|
|157|     GF 12,Gaisano Cap...|       Gaisano Capital C...|
|158|     Monday – Thursday...|              Store Hours :|
|159|     GF Tita Gwapa Sup...|       Tita Gwapa Supert...|
|160|     Monday – Sunday 7...|              Store Hours :|
+---+-------------------------+---------------------------+



In [24]:
# add an empty row at rows 155 and 58,226,237 to balance data distribution
new_row = spark.createDataFrame([("154.5","",""),("57.5","",""),("225.5","",""),("236.5","","")], ["Row","tablescraper-selected-row","tablescraper-selected-row 2"])
add_empty_row = spark.sql("SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2` FROM initial_inspect_rose_visayas").unionAll(new_row)
add_empty_row.createOrReplaceTempView("add_empty_row_rose_visayas")
new_added_row = spark.sql("""
    WITH add_empty_row AS (
        SELECT CAST(Row AS INTEGER), `tablescraper-selected-row`, `tablescraper-selected-row 2`
        FROM add_empty_row_rose_visayas
        ORDER BY `Row`
    ), 
    reorder_row AS (
        SELECT *, CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS RowCount
        FROM add_empty_row
    )
    SELECT RowCount as Row, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM reorder_row
""")

# remove the store hours from rows 
new_added_row.createOrReplaceTempView("added_empty_row_rose_visayas")
remove_store_hours_cleaning_rose_visayas = spark.sql("""
    SELECT *
    FROM added_empty_row_rose_visayas
    WHERE `Row` % 2 = 1 
""")

# separate contact from address in tablescraper-selected-row column and reorder the row count
remove_store_hours_cleaning_rose_visayas.createOrReplaceTempView("remove_store_hours_rose_visayas")
address_rose_visayas = remove_store_hours_cleaning_rose_visayas.withColumn("Address", split(remove_store_hours_cleaning_rose_visayas['tablescraper-selected-row'], '\n').getItem(0)).createOrReplaceTempView("address_rose_visayas")
contact_rose_visayas = remove_store_hours_cleaning_rose_visayas.withColumn("Contact", split(remove_store_hours_cleaning_rose_visayas['tablescraper-selected-row'], '\n').getItem(1)).createOrReplaceTempView("contact_rose_visayas")
address_contact_cleaning_rose_visayas = spark.sql("""
        SELECT 
            CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, 
            Address, 
            TRIM('Tel. ' FROM Contact) AS Contact
        FROM address_rose_visayas
        JOIN contact_rose_visayas
        ON address_rose_visayas.Row = contact_rose_visayas.Row
""")

# fill null values with 0 
fill_null_cleaning_rose_visayas = address_contact_cleaning_rose_visayas.rdd.map(lambda row: (row[0], row[1], 0 if row[2] == None else row[2]))
fill_null_cleaning_rose_visayas.collect()

# fix outlier - replace empty row with its proper value  
replace_empty_row_rose_visayas = fill_null_cleaning_rose_visayas.toDF(["Row","Address","Contact"]).replace("","Kagodoy, Basak-Marigondon Rd, Lapu-Lapu, 6015 Lalawigan ng Cebu","Address")

# final inspection for rosepharmacy - visayas
cleaned_rose_visayas = replace_empty_row_rose_visayas
cleaned_rose_visayas.show()

+---+--------------------+--------------+
|Row|             Address|       Contact|
+---+--------------------+--------------+
|  1|Osmeña Boulevard,...|(032) 230-5012|
|  2|Cinco Centrum, Fu...|(032) 253-4080|
|  3|Unit 159, Upper G...|(032) 232-0608|
|  4|Stall 277, 2nd Fl...|(032) 233-7844|
|  5|Robinsons Cyberga...|(032) 505-5921|
|  6|F Ramos Street, C...|(032) 254-7644|
|  7|Diaz Bldg. Cor. A...|(032) 230-5013|
|  8|Stall 3 & 4, G/F ...|(032) 231-9733|
|  9|V. Rama Avenue, G...|(032) 254-9544|
| 10|Piazza Elesia Com...|(032) 344-6771|
| 11|H. Cortes corner ...|(032) 230-5017|
| 12|J Center Mall,#16...|(032) 520-3608|
| 13|19-20 GF One Pavi...|(032) 236-9477|
| 14|Express Inn, J. L...|(032) 236-0154|
| 15|GF Gaisano Grand ...|(032) 401-0182|
| 16|Gov. M. Cuenco Av...|(032) 239-4153|
| 17|Anitas Bldg., cor...|(032) 239-0901|
| 18|Square 1 building...|(032) 236-6200|
| 19|Fooda Saversmart,...|(032) 260-7310|
| 20|G/F-5, Ground Flo...|(032) 262-0082|
+---+--------------------+--------

#### TRANSFORM rosepharmacy - mindanao

In [25]:
# quick inspection of rosepharmacy - luzon's dataframe
df_rose_mindanao.createOrReplaceTempView("rose_mindanao")
initial_inspect = spark.sql("""
    SELECT 
        *, 
        ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row 
    FROM rose_mindanao
""")

initial_inspect.show(20)

+-------------------------+---------------------------+---+
|tablescraper-selected-row|tablescraper-selected-row 2|Row|
+-------------------------+---------------------------+---+
|     J. Borja cor. Gui...|       Rose Pharmacy Gui...|  1|
|     Cruz Taal cor. Ap...|       Rose Pharmacy Cru...|  2|
|     R.N. Abejuela St....|       Rose Pharmacy Abe...|  3|
|     Tirso Neri/Velez ...|       Rose Pharmacy Tir...|  4|
|     Gaisano City, Cor...|       Rose Pharmacy Gai...|  5|
|     Matilde Neri cor....|       Rose Pharmacy Car...|  6|
|     Level 1, Robinson...|       Rose Pharmacy Rob...|  7|
|     Lapasan Highway, ...|       Rose Pharmacy Lap...|  8|
|     J. Borja St. cor....|       Rose Pharmacy J. ...|  9|
|     J.R. Borja cor. D...|       Rose Pharmacy Dau...| 10|
|     Gusa, Cagayan de ...|       Rose Pharmacy Gus...| 11|
|     Puerto, Cagayan d...|       Rose Pharmacy Pue...| 12|
|     T. Saco St. cor. ...|       Rose Pharmacy Mac...| 13|
|     G/F Space F. Pela...|       Rose P

In [26]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_rose_mindanao.select([count(when(col(c).isNull(), c)).alias(c) for c in df_rose_mindanao.columns])
nan_values = df_rose_mindanao.select([count(when(isnan(c), c)).alias(c) for c in df_rose_mindanao.columns])
empty_values = df_rose_mindanao.select([count(when(col(c) == "", c)).alias(c) for c in df_rose_mindanao.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+-------------------------+---------------------------+
|tablescraper-selected-row|tablescraper-selected-row 2|
+-------------------------+---------------------------+
|                        1|                          1|
|                        0|                          0|
|                        0|                          0|
+-------------------------+---------------------------+



In [27]:
# inspect further the null row 
# find the row number for the null row 
initial_inspect.createOrReplaceTempView("initial_inspect_rose_mindanao")
further_inspect = spark.sql("""
    SELECT `Row`, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM initial_inspect_rose_mindanao
    WHERE `tablescraper-selected-row` IS NULL OR `tablescraper-selected-row 2` IS NULL
""")

further_inspect.show()

+---+-------------------------+---------------------------+
|Row|tablescraper-selected-row|tablescraper-selected-row 2|
+---+-------------------------+---------------------------+
| 93|                     null|                       null|
+---+-------------------------+---------------------------+



In [28]:
# remove rows with null values in all columns 
remove_null_rows_cleaning_rose_mindanao = spark.sql("""
    SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, `tablescraper-selected-row`, `tablescraper-selected-row 2`
    FROM rose_mindanao
    WHERE `tablescraper-selected-row` IS NOT NULL OR `tablescraper-selected-row 2` IS NOT NULL
""")

# separate contact from address in tablescraper-selected-row column
remove_null_rows_cleaning_rose_mindanao.createOrReplaceTempView("remove_null_rows_rose_mindanao")
address_rose_mindanao = remove_null_rows_cleaning_rose_mindanao.withColumn("Address", split(remove_null_rows_cleaning_rose_mindanao['tablescraper-selected-row'], '\n').getItem(0)).createOrReplaceTempView("address_rose_mindanao")
contact_rose_mindanao = remove_null_rows_cleaning_rose_mindanao.withColumn("Contact", split(remove_null_rows_cleaning_rose_mindanao['tablescraper-selected-row'], '\n').getItem(1)).createOrReplaceTempView("contact_rose_mindanao")
address_contact_cleaning_rose_mindanao = spark.sql("""
        SELECT 
            CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, 
            Address, 
            Contact
        FROM address_rose_mindanao
        JOIN contact_rose_mindanao
        ON address_rose_mindanao.Row = contact_rose_mindanao.Row
""")
address_contact_cleaning_rose_mindanao = address_contact_cleaning_rose_mindanao.withColumn('Contact_tmp', regexp_extract('Address', '\\d+\\s+\\d+-\\d+\\s*/*\\s*\\d*-*\\d*', 0)).select(
    'Row',
    expr('replace(Address, Contact_tmp, "")').alias('Address'),
    coalesce('Contact', 'Contact_tmp').alias('Contact')
)

# fill empty values with 0 
fill_null_cleaning_rose_mindanao = address_contact_cleaning_rose_mindanao.rdd.map(lambda row: (row[0], row[1], 0 if row[2] == "" else row[2]))
fill_null_cleaning_rose_mindanao.collect()

# fix outlier - replace weird row with its proper value 
outlier_cleaning_rose_mindanao = fill_null_cleaning_rose_mindanao.toDF(["Row","Address","Contact"])
outlier_cleaning_rose_mindanao.createOrReplaceTempView("outlier_cleaning")
outlier_cleaning_rose_mindanao = spark.sql("""
    SELECT 
        Row,
        CASE
            WHEN Address = '839-5175'
            THEN 'GF-06 Gaisano Cap. San Jose, National Rd., Labangan, Occidental Mindoro, San Jose, 5100'
            ELSE Address
        END As Address,
        Contact
    FROM outlier_cleaning
""")
outlier_cleaning_rose_mindanao.createOrReplaceTempView("further_outlier_cleaning")
further_outlier_cleaning_rose_mindanao = spark.sql("""
    WITH union_cte AS (
        SELECT *
        FROM further_outlier_cleaning

        UNION

        SELECT 
            Row, 
            Address, 
            CASE 
                WHEN Contact = 0
                THEN '839-5175'
                ELSE Contact
            END AS Contact
        FROM further_outlier_cleaning
        WHERE Address = 'GF-06 Gaisano Cap. San Jose, National Rd., Labangan, Occidental Mindoro, San Jose, 5100'
    )
    SELECT * 
    FROM union_cte
    WHERE 
        NOT (Address = 'GF-06 Gaisano Cap. San Jose, National Rd., Labangan, Occidental Mindoro, San Jose, 5100' 
        AND Contact = "0")
    ORDER BY Row
""")

# final inspection for rosepharmacy - mindanao
cleaned_rose_mindanao = further_outlier_cleaning_rose_mindanao
cleaned_rose_mindanao.show()

+---+--------------------+--------------------+
|Row|             Address|             Contact|
+---+--------------------+--------------------+
|  1|J. Borja cor. Gui...|08822 722-922 / 7...|
|  2|Cruz Taal cor. Ap...|       08822 725-301|
|  3|R.N. Abejuela St....|       08822 727-864|
|  4|Tirso Neri/Velez ...|       08822 728-180|
|  5|Gaisano City, Cor...|        088 856-6816|
|  6|Matilde Neri cor....|       08822 726-156|
|  7|Level 1, Robinson...|        088 856-1262|
|  8|Lapasan Highway, ...|       08822 714-067|
|  9|J. Borja St. cor....|       08822 714-732|
| 10|J.R. Borja cor. D...|       08822 714-816|
| 11|Gusa, Cagayan de ...|       08822 733-840|
| 12|Puerto, Cagayan d...|        088 880-7349|
| 13|T. Saco St. cor. ...|       08822 722-204|
| 14|G/F Space F. Pela...|        088 858-2358|
| 15|Gateway Tower 1, ...|        088 852-1258|
| 16|Market City bus T...|        088 857-4022|
| 17|National Highway ...|       08822 727-061|
| 18|Gaisano Bulua , Z...|        088 88

#### Merge multiple rosepharmacy dataframes

In [29]:
# merge different dataframes 
cleaned_rose_luzon_visayas = cleaned_rose_luzon.union(cleaned_rose_visayas)
cleaned_rose = cleaned_rose_luzon_visayas.union(cleaned_rose_mindanao)
cleaned_rose.createOrReplaceTempView("cleaned_rose")
cleaned_rose = spark.sql("""
    SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, Address, Contact
    FROM cleaned_rose
""")

# initial and final inspect since these data are already cleaned
cleaned_rose = cleaned_rose.withColumn("Name", lit("Rose Pharmacy"))
cleaned_rose.show(5)
cleaned_rose.printSchema()

# show Null, NaN, and Empty Values
null_values = cleaned_rose.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_rose.columns])
nan_values = cleaned_rose.select([count(when(isnan(c), c)).alias(c) for c in cleaned_rose.columns])
empty_values = cleaned_rose.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_rose.columns])
append = null_values.union(nan_values).union(empty_values)
append.show()

# show basic statistics
cleaned_rose.describe().show()

+---+--------------------+-------------+-------------+
|Row|             Address|      Contact|         Name|
+---+--------------------+-------------+-------------+
|  1|1st Flr, Festival...|(02) 850-2853|Rose Pharmacy|
|  2|Zapote Road, Fili...|(02) 807-9102|Rose Pharmacy|
|  3|G/F Asian Hospita...|(02) 659-3394|Rose Pharmacy|
|  4|space 2102A & 210...|(02) 822-9492|Rose Pharmacy|
|  5|   Sucat, Muntinlupa|(02) 856-0293|Rose Pharmacy|
+---+--------------------+-------------+-------------+
only showing top 5 rows

root
 |-- Row: integer (nullable = false)
 |-- Address: string (nullable = true)
 |-- Contact: string (nullable = true)
 |-- Name: string (nullable = false)

+---+-------+-------+----+
|Row|Address|Contact|Name|
+---+-------+-------+----+
|  0|      0|      0|   0|
|  0|      0|      0|   0|
|  0|      0|      0|   0|
+---+-------+-------+----+

+-------+----------------+--------------------+--------------------+-------------+
|summary|             Row|             Address|  

#### TRANSFORM mercury drugs - luzon

In [30]:
# quick inspection of mercury drugs - luzon's dataframe
df_mercury_luzon.createOrReplaceTempView("mercury_luzon")
initial_inspect = spark.sql("""
    SELECT  *
    FROM mercury_luzon
""")

initial_inspect.show(20)

+--------------------+--------------------+--------------------+--------------------+
|                Name|             Address|               Phone|         Store Hours|
+--------------------+--------------------+--------------------+--------------------+
|     Bangued Partelo|\n Partelo St. \n...|\n (074) 752-8152...|\n 6:00 AM - 7:00...|
|        Bangued Taft|\n    \n Taft Ave...|\n (074) 752-5852...|\n 6:00 AM - 9:00...|
|   Bacacay Poblacion|\n SSBF Bldg. \n ...|           \n () \n |\n 7:00 AM - 8:00...|
|  Daraga B.U. Arcade|\n Bicol Universi...|\n (052) 483-4151...|\n 6:00 AM - 10:0...|
|Legazpi City Ayal...|\n Ayala Malls Le...| \n (052) 2010464\n |\n 10:00 AM - 8:0...|
|Legazpi City Old ...|\n GTS Bldg., \n ...|\n (052) 480-5129\n |\n 6:00 AM - 11:0...|
|  Legazpi City Rizal|\n Del Rosario Bl...|\n (052) 480-5136\n |\n 6:00 AM - 10:0...|
|Legazpi City Wash...|\n Rivero Bldg., ...|\n (052) 481-0988\n |\n 6:00 AM - 10:0...|
| Ligao City McKinley|\n - \n McKinley ...|\n (052) 48

In [31]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_mercury_luzon.select([count(when(col(c).isNull(), c)).alias(c) for c in df_mercury_luzon.columns])
nan_values = df_mercury_luzon.select([count(when(isnan(c), c)).alias(c) for c in df_mercury_luzon.columns])
empty_values = df_mercury_luzon.select([count(when(col(c) == "", c)).alias(c) for c in df_mercury_luzon.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+----+-------+-----+-----------+
|Name|Address|Phone|Store Hours|
+----+-------+-----+-----------+
|   0|      0|    0|          0|
|   0|      0|    0|          0|
|   0|      0|    0|          0|
+----+-------+-----+-----------+



In [32]:
# remove name and store hours, trim properly the attributes in each column
remove_columns_cleaning_mercury_luzon = spark.sql("""
    WITH first_trim AS (
        SELECT 
            ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row,
            REPLACE(Address, "\n", "") AS Address,
            REPLACE(Phone, "\n", "") AS Contact
        FROM mercury_luzon
    ), second_trim AS (
        SELECT 
            Row,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Address),"  ", " "), "  ", " "),"  ", " "), "- ", "") As Address,
            TRIM(Contact) AS Contact
        FROM first_trim
    )
    SELECT * 
    FROM second_trim
""")

# final inspection for mercury - luzon
cleaned_mercury_luzon = remove_columns_cleaning_mercury_luzon 
cleaned_mercury_luzon.show()

+---+--------------------+--------------------+
|Row|             Address|             Contact|
+---+--------------------+--------------------+
|  1|Partelo St. Zone ...|(074) 752-8152 / ...|
|  2|Taft Ave. Zone V ...|(074) 752-5852 / ...|
|  3|SSBF Bldg. Mora S...|                  ()|
|  4|Bicol University ...|(052) 483-4151 / ...|
|  5|Ayala Malls Legaz...|       (052) 2010464|
|  6|GTS Bldg., Rizal ...|      (052) 480-5129|
|  7|Del Rosario Bldg....|      (052) 480-5136|
|  8|Rivero Bldg., Was...|      (052) 481-0988|
|  9|McKinley St., Pur...|      (052) 485-1150|
| 10|LCC Mall Polangui...|                  ()|
| 11|Ubaliw, Polangui ...|      (052) 486-1150|
| 12|Ziga Ave. cor. Bo...|(052) 487-7010 / ...|
| 13|Quezon St. Baler ...|(042) 722-0008 / ...|
| 14|Paterno St. Balan...|(047) 237-5269 / ...|
| 15|Public Market Bal...|(047) 791-1818 / ...|
| 16|Rizal St. Balanga...|(047) 237-5270 / ...|
| 17|Roman Superhway T...|      (047) 633-3658|
| 18|G/F WBLN 042 Walt...|              

#### TRANSFORM mercury drugs - visayas

In [33]:
# quick inspection of mercury drugs - visayas' dataframe
df_mercury_visayas.createOrReplaceTempView("mercury_visayas")
initial_inspect = spark.sql("""
    SELECT  *
    FROM mercury_visayas
""")

initial_inspect.show(20)

+--------------------+--------------------+--------------------+--------------------+
|                Name|             Address|               Phone|         Store Hours|
+--------------------+--------------------+--------------------+--------------------+
|    Ibajay Poblacion|\n Sia-Chan Lek B...|\n (036) 289-2774...|\n 7:00 AM - 8:00...|
|      Kalibo Andagao|\n La Estrella Bl...|\n (036) 268-4401...|\n 7:00 AM - 9:00...|
|Kalibo Archbishop...|\n               ...|\n (036) 268-4123...|\n 7:00 AM - 9:00...|
|      Kalibo Barrios|\n Roxas Ave.cor....|\n (036) 268-6316...|\n 7:00 AM - 9:30...|
|       Kalibo Mabini|\n Mabini St. \n ...|\n (036) 500-7899...|\n 8:00 AM - 10:0...|
|        Kalibo Roxas|\n Roxas Ave. Ext...|\n (036) 268-5143...|\n 7:00 AM - 8:00...|
|Malay Caticlan Je...|\n   \n  Jetty Po...|\n (036) 288-7629...|\n 7:00 AM - 9:00...|
|Culasi Centro Pob...|\n Godo Yap Bldg....|\n (036) 277-8718...|\n 7:00 AM - 8:00...|
|San Jose De Buena...|\n  Tobias Fornie...|\n (036) 54

In [34]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_mercury_visayas.select([count(when(col(c).isNull(), c)).alias(c) for c in df_mercury_visayas.columns])
nan_values = df_mercury_visayas.select([count(when(isnan(c), c)).alias(c) for c in df_mercury_visayas.columns])
empty_values = df_mercury_visayas.select([count(when(col(c) == "", c)).alias(c) for c in df_mercury_visayas.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+----+-------+-----+-----------+
|Name|Address|Phone|Store Hours|
+----+-------+-----+-----------+
|   0|      0|    0|          0|
|   0|      0|    0|          0|
|   0|      0|    0|          0|
+----+-------+-----+-----------+



In [35]:
# remove name and store hours, trim properly the attributes in each column
remove_columns_cleaning_mercury_visayas = spark.sql("""
    WITH first_trim AS (
        SELECT 
            ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row,
            REPLACE(Address, "\n", "") AS Address,
            REPLACE(Phone, "\n", "") AS Contact
        FROM mercury_visayas
    ), second_trim AS (
        SELECT 
            Row,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Address),"  ", " "), "  ", " "),"  ", " "), "- ", "") AS Address,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Contact),"  ", " "), "  ", " "),"  ", " "),"  ", " ") AS Contact
        FROM first_trim
    )
    SELECT * 
    FROM second_trim
""")

# final inspection for mercury - visayas
cleaned_mercury_visayas = remove_columns_cleaning_mercury_visayas 
cleaned_mercury_visayas.show()

+---+--------------------+--------------------+
|Row|             Address|             Contact|
+---+--------------------+--------------------+
|  1|Sia-Chan Lek Bldg...|(036) 289-2774 / ...|
|  2|La Estrella Bldg....|(036) 268-4401 / ...|
|  3|Archbishop Reyes ...|(036) 268-4123 / ...|
|  4|Roxas Ave.cor. L....|(036) 268-6316 / ...|
|  5|Mabini St. Kalibo...|(036) 500-7899 / ...|
|  6|Roxas Ave. Ext. K...|(036) 268-5143 / ...|
|  7|Jetty Port , Cati...|(036) 288-7629 / ...|
|  8|Godo Yap Bldg. Na...|(036) 277-8718 / ...|
|  9|Tobias Fornier St...|(036) 540-9727 / ...|
| 10|Emigdio Nietes Av...|(036) 641-6043 / ...|
| 11|Aguillon Business...|( ) / (036) 64100...|
| 12|Gov. Villavert St...|(036) 540-8048 / ...|
| 13|HPL bldg. Venegas...|( ) / (036) 543-8...|
| 14|P. Inocentes St. ...|( ) / (053) 500-3...|
| 15|Sta. Cruz St. Pag...|(038) 238-3182 / ...|
| 16|J.A Clarin St. Co...|(038) 412-0951 / ...|
| 17|B. Inting St. cor...|(038) 411-3087 / ...|
| 18|C.P. Garcia North...|(038) 411-1365

#### TRANSFORM mercury drugs - mindanao

In [36]:
# quick inspection of mercury drugs - mindanao' dataframe
df_mercury_mindanao.createOrReplaceTempView("mercury_mindanao")
initial_inspect = spark.sql("""
    SELECT  *
    FROM mercury_mindanao
""")

initial_inspect.show(20)

+--------------------+--------------------+--------------------+--------------------+
|                Name|             Address|               Phone|         Store Hours|
+--------------------+--------------------+--------------------+--------------------+
|Butuan  City Satorre|\n    \n Jorge Sa...|\n (085) 341-4024...|\n 7:00 AM - 10:0...|
|  Butuan City Curato|\n A.D. Curato St...|\n (085) 815-1855...|\n 7:00 AM - 10:0...|
|Butuan City J.C. ...|\n          \n J....|\n (085) 815-6457...|\n 7:00 AM - 10:0...|
|Butuan City Libertad|\n Helen Dy Bldg....|\n (085) 341-7009...|\n 7:00 AM - 9:00...|
|Butuan City Monti...|\n Montilla Blvd....|\n (085) 342-6888...|\n 8:00 AM - 10:0...|
|Butuan City Robin...|\n Robinsons Plac...|\n (085) 815-0758...|\n 10:00 AM - 9:0...|
|Butuan City South...|\n South Blvd. Sq...|\n (085) 341-3928...|\n 7:00 AM - 10:0...|
|Cabadbaran City N...|\n Nat'l Hway  \n...|\n (085) 818-2323...|\n 7:00 AM - 8:00...|
|Bayugan City Gais...|\n Unit No. BYN-G...|   \n (   )

In [37]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_mercury_visayas.select([count(when(col(c).isNull(), c)).alias(c) for c in df_mercury_visayas.columns])
nan_values = df_mercury_visayas.select([count(when(isnan(c), c)).alias(c) for c in df_mercury_visayas.columns])
empty_values = df_mercury_visayas.select([count(when(col(c) == "", c)).alias(c) for c in df_mercury_visayas.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+----+-------+-----+-----------+
|Name|Address|Phone|Store Hours|
+----+-------+-----+-----------+
|   0|      0|    0|          0|
|   0|      0|    0|          0|
|   0|      0|    0|          0|
+----+-------+-----+-----------+



In [38]:
# remove name and store hours, trim properly the attributes in each column
remove_columns_cleaning_mercury_mindanao = spark.sql("""
    WITH first_trim AS (
        SELECT 
            ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row,
            REPLACE(Address, "\n", "") AS Address,
            REPLACE(Phone, "\n", "") AS Contact
        FROM mercury_mindanao
    ), second_trim AS (
        SELECT 
            Row,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Address),"  ", " "), "  ", " "),"  ", " "), "- ", "") AS Address,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Contact),"  ", " "), "  ", " "),"  ", " "),"  ", " ") AS Contact
        FROM first_trim
    )
    SELECT * 
    FROM second_trim
""")

# final inspection for mercury - mindanao
cleaned_mercury_mindanao = remove_columns_cleaning_mercury_mindanao
cleaned_mercury_mindanao.show()

+---+--------------------+--------------------+
|Row|             Address|             Contact|
+---+--------------------+--------------------+
|  1|Jorge Satorre St....|(085) 341-4024 / ...|
|  2|A.D. Curato St. B...|(085) 815-1855 / ...|
|  3|J.C. Aquino Ave. ...|(085) 815-6457 / ...|
|  4|Helen Dy Bldg. Na...|(085) 341-7009 / ...|
|  5|Montilla Blvd. co...|(085) 342-6888 / ...|
|  6|Robinsons Place B...|(085) 815-0758 / ...|
|  7|South Blvd. Squar...|(085) 341-3928 / ...|
|  8|Nat'l Hway Cabadb...|(085) 818-2323 / ...|
|  9|Unit No. BYN-GPS-...|                 ( )|
| 10|Narra Ave. cor. C...|(085) 830-5535 / ...|
| 11|None Nat'l Hway P...|(085) 343-6988 / ...|
| 12|Gaisano Grand Mal...|(085) 839-5631 / ...|
| 13|Nat'l Hway Purok ...|(085) 343-8134 / ...|
| 14|None Roxas St. Sa...|(085) 839-3410 / ...|
| 15|Nat'l Hway Poblac...|(085) 255-2930 / ...|
| 16|Fortich St. Malay...|(088) 221-2129 / ...|
| 17|Sebastian St. cor...|(088) 813-4800 / ...|
| 18|W.G. Bldg. Purok ...|(088) 828-3997

#### TRANSFORM mercury drugs - metro manila

In [39]:
# quick inspection of mercury drugs - mindanao' dataframe
df_mercury_metro_manila.createOrReplaceTempView("mercury_metro_manila")
initial_inspect = spark.sql("""
    SELECT  *
    FROM mercury_metro_manila
""")

initial_inspect.show(20)

+--------------------+--------------------+--------------------+--------------------+
|                Name|             Address|               Phone|         Store Hours|
+--------------------+--------------------+--------------------+--------------------+
|Kalookan City 11t...|\n    \n Rizal Av...|\n (02) 8-366-177...|\n 6:00 AM - 10:0...|
|Kalookan City 5th...|\n Rizal Ave. Ext...|\n (02) 8-361-799...|\n 6:00 AM - 8:00...|
|Kalookan City 9th...|\n Aurelio Bldg. ...|\n (02) 8-361-120...|\n 6:00 AM - 9:00...|
|Kalookan City Bag...|\n Ph 2 Blk 2 L3 ...|\n (02) 8-962-371...|\n 6:00 AM - 8:00...|
|Kalookan City Ban...|\n     \n Blk 2 L...|\n (02) 8-330-450...|\n 6:00 AM - 8:00...|
|   Kalookan City C-3|\n    \n A. Mabin...|\n (02) 8-285-946...|\n 6:00 AM - 10:0...|
|Kalookan City Cam...|\n 2194 \n Zapote...|\n (02) 8-961-162...|\n 6:00 AM - 8:00...|
| Kalookan City iMall|\n iMall Camarin ...|\n (02) 8-542 865...|\n 6:00 AM - 9:00...|
|Kalookan City Kay...|\n 13 Gen. Luis S...|\n (02) 8-9

In [40]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_mercury_visayas.select([count(when(col(c).isNull(), c)).alias(c) for c in df_mercury_visayas.columns])
nan_values = df_mercury_visayas.select([count(when(isnan(c), c)).alias(c) for c in df_mercury_visayas.columns])
empty_values = df_mercury_visayas.select([count(when(col(c) == "", c)).alias(c) for c in df_mercury_visayas.columns])

append = null_values.union(nan_values).union(empty_values)
append.show()

+----+-------+-----+-----------+
|Name|Address|Phone|Store Hours|
+----+-------+-----+-----------+
|   0|      0|    0|          0|
|   0|      0|    0|          0|
|   0|      0|    0|          0|
+----+-------+-----+-----------+



In [41]:
# remove name and store hours, trim properly the attributes in each column
remove_columns_cleaning_mercury_metro_manila = spark.sql("""
    WITH first_trim AS (
        SELECT 
            ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS Row,
            REPLACE(Address, "\n", "") AS Address,
            REPLACE(Phone, "\n", "") AS Contact
        FROM mercury_metro_manila
    ), second_trim AS (
        SELECT 
            Row,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Address),"  ", " "), "  ", " "),"  ", " "), "- ", "") AS Address,
            REPLACE(REPLACE(REPLACE(REPLACE(TRIM(Contact),"  ", " "), "  ", " "),"  ", " "),"  ", " ") AS Contact
        FROM first_trim
    )
    SELECT * 
    FROM second_trim
""")

# final inspection for mercury - metro_manila
cleaned_mercury_metro_manila = remove_columns_cleaning_mercury_metro_manila
cleaned_mercury_metro_manila.show()

+---+--------------------+--------------------+
|Row|             Address|             Contact|
+---+--------------------+--------------------+
|  1|Rizal Ave. Ext. c...|(02) 8-366-1774 /...|
|  2|Rizal Ave. Ext. b...|(02) 8-361-7994 /...|
|  3|Aurelio Bldg. Riz...|(02) 8-361-1206 /...|
|  4|Ph 2 Blk 2 L3 Bag...|(02) 8-962-3712 /...|
|  5|Blk 2 L11, Quirin...|(02) 8-330-4509 /...|
|  6|A. Mabini St. cor...|(02) 8-285-9463 /...|
|  7|2194 Zapote St. c...|(02) 8-961-1620 /...|
|  8|iMall Camarin Cam...|(02) 8-542 8659 /...|
|  9|13 Gen. Luis St. ...|(02) 8-983-9975 /...|
| 10|J.P. Rizal St. co...|(02) 8-285-9488 /...|
| 11|Bonifacio Monumen...|(02) 8-361-6660 /...|
| 12|A. Mabini St. cor...|(02) 8-288-0032 /...|
| 13|1282 Samson Rd. B...|(02) 8-364-2393 /...|
| 14|Puregold Kalookan...|(02) 8-709-6793 /...|
| 15|78 Gen. San Migue...|(02) 8-285-6461 /...|
| 16|People's Place Bl...|(02) 8-287 1209 /...|
| 17|Zabarte Town Cent...|(02) 8-962-3967 /...|
| 18|Alabang Zapote Rd...|(02) 8-800-074

#### Merge multiple mercury drugs dataframe

In [42]:
# merge different dataframes 
cleaned_mercury = cleaned_mercury_luzon.union(cleaned_mercury_metro_manila).union(cleaned_mercury_visayas).union(cleaned_mercury_mindanao)
cleaned_mercury.createOrReplaceTempView("cleaned_mercury")
cleaned_mercury = spark.sql("""
    SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, Address, Contact
    FROM cleaned_mercury
""")

# initial and final inspect since these data are already cleaned
cleaned_mercury = cleaned_mercury.withColumn("Name", lit("Mercury Drugs"))
cleaned_mercury.show(5)
cleaned_mercury.printSchema()

# show Null, NaN, and Empty Values
null_values = cleaned_mercury.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_mercury.columns])
nan_values = cleaned_mercury.select([count(when(isnan(c), c)).alias(c) for c in cleaned_mercury.columns])
empty_values = cleaned_mercury.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_mercury.columns])
append = null_values.union(nan_values).union(empty_values)
append.show()

# show basic statistics
cleaned_mercury.describe().show()

+---+--------------------+--------------------+-------------+
|Row|             Address|             Contact|         Name|
+---+--------------------+--------------------+-------------+
|  1|Partelo St. Zone ...|(074) 752-8152 / ...|Mercury Drugs|
|  2|Taft Ave. Zone V ...|(074) 752-5852 / ...|Mercury Drugs|
|  3|SSBF Bldg. Mora S...|                  ()|Mercury Drugs|
|  4|Bicol University ...|(052) 483-4151 / ...|Mercury Drugs|
|  5|Ayala Malls Legaz...|       (052) 2010464|Mercury Drugs|
+---+--------------------+--------------------+-------------+
only showing top 5 rows

root
 |-- Row: integer (nullable = false)
 |-- Address: string (nullable = true)
 |-- Contact: string (nullable = true)
 |-- Name: string (nullable = false)

+---+-------+-------+----+
|Row|Address|Contact|Name|
+---+-------+-------+----+
|  0|      0|      0|   0|
|  0|      0|      0|   0|
|  0|      0|      0|   0|
+---+-------+-------+----+

+-------+------------------+--------------------+--------------------

#### Transform south star drugs 

In [43]:
# quick inspection of south star drugs - name's dataframe
df_ssd_others.createOrReplaceTempView("ssd_others")
initial_inspect = spark.sql("""
    SELECT  *
    FROM ssd_others
""")

initial_inspect.show(20, truncate=False)

+--------------------------------------------------------------------------+
|Address_StoreHours_Phone_RemoveHrTag                                      |
+--------------------------------------------------------------------------+
|Ph 148-01 Metro Plaza Mall, Quirino Highway, Malaria, Caloocan City       |
|Store Hours: 7:00AM - 9:00PM                                              |
|<hr class="dashed"/>                                                      |
|Ground Floor Victory Central Mall, Caloocan City                          |
|+632-82833778                                                             |
|Store Hours:                                                              |
|<hr class="dashed"/>                                                      |
|Phase 1 Package 1 Block 28 Lot 3638 Bagong Silang Caloocan City           |
|+632-83722847                                                             |
|Store Hours: 7:00AM-7:00PM                                                |

In [44]:
# show additional information - total Null, NaN, and Empty Values
null_values = df_ssd_others.select([count(when(col(c).isNull(), c)).alias(c) for c in df_ssd_others.columns])
nan_values = df_ssd_others.select([count(when(isnan(c), c)).alias(c) for c in df_ssd_others.columns])
empty_values = df_ssd_others.select([count(when(col(c) == "", c)).alias(c) for c in df_ssd_others.columns])
zero_values = df_ssd_others.select([count(when(col(c) == "0", c)).alias(c) for c in df_ssd_others.columns])

append = null_values.union(nan_values).union(empty_values).union(zero_values)
append.show()

+------------------------------------+
|Address_StoreHours_Phone_RemoveHrTag|
+------------------------------------+
|                                   0|
|                                   0|
|                                   0|
|                                   0|
+------------------------------------+



In [45]:
# fix outlier - row 1949 and row 1950 should be merged 
remove_outlier_cleaning_ssd = spark.sql("""
    WITH create_row AS (
        SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, * 
        FROM ssd_others
    ), remove_outlier AS (
        SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, Address_StoreHours_Phone_RemoveHrTag
        FROM create_row 
        WHERE Row != 1950
    )
    SELECT 
        Row,
        CASE
            WHEN Row = 1949 THEN "PH. 31101, Robinsons Supermarket Mactan, Mez 2, Pueblo Verde Subd., Brgy. Basak, Lapulapu City, Cecu"
            ELSE Address_StoreHours_Phone_RemoveHrTag
        END as ssd_others
    FROM remove_outlier
    
""")
remove_outlier_cleaning_ssd.createOrReplaceTempView("remove_outlier_ssd")

# separate address, store hours, hr, and contact
ssd_cleaning = spark.sql("""
    SELECT *
    FROM remove_outlier_ssd
    WHERE NOT ssd_others LIKE "SSD%"
""")

# 1. convert rows into list where you could iterate from -> iterate thru each of the rows if filtered_hr_tag is true then add empty row at current index - 2, reorder the row at every iteration
ssd_row = ssd_cleaning.select("Row").rdd.flatMap(lambda x: x).collect()
ssd_other_info = ssd_cleaning.select("ssd_others").rdd.flatMap(lambda x: x).collect()

ssd_list = [list(x) for x in zip(ssd_row, ssd_other_info)]

for row in ssd_list:
    if (row[1] == '<hr class="dashed"/>' or row[1] == '<br/>') and row[0] % 4 != 0:
        new_empty_row = [row[0] - 1.5,"0"]
        ssd_list.append(new_empty_row) 
        ssd_list.sort(key = lambda x: x[0])
        for index,x in enumerate(ssd_list):
            ssd_list[index][0] = index+1      
            
reordered_data_distribution_cleaning_ssd = spark.createDataFrame(ssd_list)
reordered_data_distribution_cleaning_ssd.createOrReplaceTempView("reordered_data_ssd")  

pivot_ssd = spark.sql("""
    WITH reordering_data AS (
        SELECT *,
            CASE 
                WHEN _1 % 4 = 0 THEN 4
                WHEN (_1 + 1) % 4 = 0 THEN 3
                WHEN _1 % 2 = 0 THEN 2
                WHEN (_1 + 3) % 4 = 0 THEN 1
            END AS _3
        FROM reordered_data_ssd
    ), grab_address_contact AS (
        SELECT 
            CASE
                WHEN _3 = 1
                THEN _2
            END AS Address,
            CASE
                WHEN _3 = 2
                THEN _2
            END AS Contact,
            _3
        FROM reordering_data
        WHERE `_3` != 3 AND `_3` != 4
    ), select_address AS (
        SELECT 
            CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row,
            Address
        FROM grab_address_contact
        WHERE Address IS NOT NULL
    ), select_contact AS (
        SELECT 
            CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row,
            Contact
        FROM grab_address_contact
        WHERE Contact IS NOT NULL
    )
    SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, c1.Address, c2.Contact
    FROM select_address AS c1, select_contact AS c2
    WHERE c1.Row = c2.Row
""")  

cleaned_ssd = pivot_ssd
cleaned_ssd = cleaned_ssd.withColumn("Name", lit("South Star Drugs"))

In [46]:
# final inspection for ssd
cleaned_ssd.show()
cleaned_ssd.printSchema()

# show Null, NaN, and Empty Values
null_values = cleaned_ssd.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_ssd.columns])
nan_values = cleaned_ssd.select([count(when(isnan(c), c)).alias(c) for c in cleaned_ssd.columns])
empty_values = cleaned_ssd.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_ssd.columns])
append = null_values.union(nan_values).union(empty_values)
append.show()

# show basic statistics
cleaned_ssd.describe().show()

+---+--------------------+---------------+----------------+
|Row|             Address|        Contact|            Name|
+---+--------------------+---------------+----------------+
|  1|Ph 148-01 Metro P...|              0|South Star Drugs|
|  2|Ground Floor Vict...|  +632-82833778|South Star Drugs|
|  3|Phase 1 Package 1...|  +632-83722847|South Star Drugs|
|  4|#1107 General San...|  +632-89617601|South Star Drugs|
|  5|Ph1 Pkg3 B44 L33 ...|+63923-744-2165|South Star Drugs|
|  6|G/F Lc G02, Barri...|+63922-928-5112|South Star Drugs|
|  7|Door 1 Rubetan Bl...|  +632-88087626|South Star Drugs|
|  8|J. Aguilar Ave. P...|  +632-82734691|South Star Drugs|
|  9|Gf Of Saver Mart ...|  +632-88695494|South Star Drugs|
| 10|Ph 41601 G/F Robi...|  +632-88323837|South Star Drugs|
| 11|Bf Resort Drive B...|  +632-88335354|South Star Drugs|
| 12|Robinsons Superma...|  +632-82935438|South Star Drugs|
| 13|#53 Women's Club ...|  +632-83724955|South Star Drugs|
| 14|San Joaquin St., ...|  +632-8532877

#### Merge all of the dataframes(branches) from multiple drugstores

In [47]:
# merge different dataframes 
cleaned_drugstores = cleaned_mercury.union(cleaned_tgp).union(cleaned_rose).union(cleaned_generika).union(cleaned_ssd).union(cleaned_watsons)
cleaned_drugstores.createOrReplaceTempView("cleaned_drugstores")
cleaned_drugstores = spark.sql("""
    SELECT CAST(ROW_NUMBER() OVER (ORDER BY (SELECT 1)) AS INTEGER) AS Row, Name, Address, Contact
    FROM cleaned_drugstores
""")

# initial and final inspect since these data are already cleaned
cleaned_drugstores.show(5)
cleaned_drugstores.printSchema()

# show Null, NaN, and Empty Values
null_values = cleaned_drugstores.select([count(when(col(c).isNull(), c)).alias(c) for c in cleaned_drugstores.columns])
nan_values = cleaned_drugstores.select([count(when(isnan(c), c)).alias(c) for c in cleaned_drugstores.columns])
empty_values = cleaned_drugstores.select([count(when(col(c) == "", c)).alias(c) for c in cleaned_drugstores.columns])
append = null_values.union(nan_values).union(empty_values)
append.show()

# show basic statistics
cleaned_drugstores.describe().show()

+---+-------------+--------------------+--------------------+
|Row|         Name|             Address|             Contact|
+---+-------------+--------------------+--------------------+
|  1|Mercury Drugs|Partelo St. Zone ...|(074) 752-8152 / ...|
|  2|Mercury Drugs|Taft Ave. Zone V ...|(074) 752-5852 / ...|
|  3|Mercury Drugs|SSBF Bldg. Mora S...|                  ()|
|  4|Mercury Drugs|Bicol University ...|(052) 483-4151 / ...|
|  5|Mercury Drugs|Ayala Malls Legaz...|       (052) 2010464|
+---+-------------+--------------------+--------------------+
only showing top 5 rows

root
 |-- Row: integer (nullable = false)
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Contact: string (nullable = true)

+---+----+-------+-------+
|Row|Name|Address|Contact|
+---+----+-------+-------+
|  0|   0|      0|      0|
|  0|   0|      0|      0|
|  0|   0|      0|      0|
+---+----+-------+-------+

+-------+-----------------+----------------+--------------------+-----

#### Convert data into parquet 

In [48]:
cleaned_mercury.write.mode('overwrite').parquet("../data/transformed-data/cleaned_mercury.parquet")
cleaned_watsons.write.mode('overwrite').parquet("../data/transformed-data/cleaned_watsons.parquet")
cleaned_tgp.write.mode('overwrite').parquet("../data/transformed-data/cleaned_tgp.parquet")
cleaned_ssd.write.mode('overwrite').parquet("../data/transformed-data/cleaned_ssd.parquet")
cleaned_rose.write.mode('overwrite').parquet("../data/transformed-data/cleaned_rose.parquet")
cleaned_generika.write.mode('overwrite').parquet("../data/transformed-data/cleaned_generika.parquet")
cleaned_drugstores.write.mode('overwrite').parquet("../data/transformed-data/cleaned_drugstores.parquet")